In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

# Load the datasets
games_updated_df = pd.read_csv('Downloads/Games_Updated.csv')
steam_df = pd.read_csv('Downloads/steam.csv')
vgsales_df = pd.read_csv('Downloads/vgsales-2.csv')

# Extract unique game names
games_updated_names = set(games_updated_df['Game_Name'].str.lower())
steam_names = set(steam_df['Game_Name'].str.lower())
vgsales_names = set(vgsales_df['Game_Name'].str.lower())

# Function to remove years from names
def remove_years(name):
    return re.sub(r'\b(19|20)?\d{2}\b', '', name).strip()

# Apply the year removal and collect unique names
all_game_names = list(set(remove_years(name) for name in games_updated_names | steam_names | vgsales_names))

# Vectorize names using TF-IDF
vectorizer = TfidfVectorizer().fit_transform(all_game_names)
cosine_similarities = cosine_similarity(vectorizer)

# Set threshold for similarity
SIMILARITY_THRESHOLD = 0.7  # Cosine similarity threshold

# Identify similar names based on cosine similarity
similar_names = {}
for idx, name in enumerate(all_game_names):
    similar_indices = cosine_similarities[idx] >= SIMILARITY_THRESHOLD
    similar_matches = [all_game_names[i] for i in range(len(all_game_names)) if similar_indices[i] and i != idx]
    if similar_matches:
        similar_names[name] = similar_matches

# Display results
for base_name, matches in list(similar_names.items()):  # Show first 10 matches for readability
    print(f"Base Name: {base_name}")
    print("Similar Names:")
    for match in matches:
        print(f" - {match}")
    print()


Base Name: paladin
Similar Names:
 - panzer paladin
 - flight of the paladin

Base Name: realms of arkania 1 - blade of destiny classic
Similar Names:
 - realms of arkania: blade of destiny

Base Name: symphony of stars
Similar Names:
 - symphony

Base Name: raw danger
Similar Names:
 - raw danger! (jp sales)
 - raw danger!

Base Name: the rift
Similar Names:
 - dark rift
 - rift

Base Name: evil glitch
Similar Names:
 - glitch

Base Name: fate/tiger colosseum upper
Similar Names:
 - fate/tiger colosseum

Base Name: metal slug anthology
Similar Names:
 - metal slug 7
 - metal slug 4 & 5
 - metal slug
 - metal slug x
 - metal slug 2
 - metal slug 3

Base Name: blade arcus from shining ex
Similar Names:
 - blade arcus from shining: battle arena

Base Name: victoria ii
Similar Names:
 - victoria

Base Name: insane
Similar Names:
 - insane robots
 - insane 2
 - insane road

Base Name: time
Similar Names:
 - time in time
 - no time
 - battle time

Base Name: mechassault: phantom war
Similar

In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rapidfuzz import fuzz
import re
from datetime import datetime

# Load the datasets
games_updated_df = pd.read_csv('Downloads/Games_Updated.csv')
steam_df = pd.read_csv('Downloads/steam.csv')
vgsales_df = pd.read_csv('Downloads/vgsales-2.csv')

# Extract and standardize release years
def extract_year(date_str):
    try:
        return datetime.strptime(date_str, "%Y").year
    except ValueError:
        try:
            return datetime.strptime(date_str, "%Y-%m-%d").year
        except ValueError:
            try:
                return datetime.strptime(date_str, "%m/%d/%Y").year
            except ValueError:
                return None

# Add standardized year columns based on available date columns
games_updated_df['Year'] = None
if 'Release_Date' in steam_df.columns:
    steam_df['Year'] = steam_df['Release_Date'].apply(lambda x: extract_year(str(x)))
if 'Release_Year' in vgsales_df.columns:
    vgsales_df['Year'] = vgsales_df['Release_Year'].apply(lambda x: int(x) if pd.notnull(x) else None)

# Function to remove years from names
def remove_years(name):
    return re.sub(r'\b(19|20)?\d{2}\b', '', name).strip()

# Collect all names, years, and source identifiers into a unified list
games_updated_names = [(remove_years(name), year, "Games_Updated") for name, year in zip(games_updated_df['Game_Name'].str.lower(), games_updated_df['Year'])]
steam_names = [(remove_years(name), year, "Steam") for name, year in zip(steam_df['Game_Name'].str.lower(), steam_df['Year'])]
vgsales_names = [(remove_years(name), year, "VG_Sales") for name, year in zip(vgsales_df['Game_Name'].str.lower(), vgsales_df['Year'])]

# Combine all names and years with sources
all_game_names = list(set(games_updated_names + steam_names + vgsales_names))

# Vectorize the game names only (for matching by name)
vectorizer = TfidfVectorizer().fit_transform([name for name, year, source in all_game_names])
cosine_similarities = cosine_similarity(vectorizer)

# Set high threshold for cosine similarity and token-based similarity
COSINE_THRESHOLD = 0.8
TOKEN_THRESHOLD = 90

# Identify similar names based on cosine similarity, token similarity, same year, and different source
similar_names = {}
for idx, (name, year, source) in enumerate(all_game_names):
    similar_indices = cosine_similarities[idx] >= COSINE_THRESHOLD
    for i in range(len(all_game_names)):
        # Check if cosine similarity is above threshold, token similarity matches, years match, and sources differ
        if (
            similar_indices[i] and i != idx and all_game_names[i][1] == year 
            and all_game_names[i][2] != source
        ):
            token_similarity = fuzz.token_sort_ratio(name, all_game_names[i][0])
            word_count_difference = abs(len(name.split()) - len(all_game_names[i][0].split()))
            
            if token_similarity >= TOKEN_THRESHOLD and word_count_difference <= 1:
                if name not in similar_names:
                    similar_names[name] = []
                similar_names[name].append((all_game_names[i][0], all_game_names[i][1], all_game_names[i][2], cosine_similarities[idx, i]))

# Display results sorted by similarity score
for base_name, matches in list(similar_names.items()):  # Show first 10 matches for readability
    print(f"Base Name: {base_name}")
    print("Similar Names:")
    for match_name, match_year, match_source, similarity in sorted(matches, key=lambda x: -x[3]):  # Sort by descending similarity
        print(f" - {match_name} (Year: {match_year}, Source: {match_source}, Similarity: {similarity:.2f})")
    print()


Base Name: batman: arkham origins
Similar Names:
 - batman™: arkham origins (Year: 2013, Source: Steam, Similarity: 1.00)

Base Name: the darkness ii
Similar Names:
 - the darkness ii (Year: 2012.0, Source: VG_Sales, Similarity: 1.00)
 - the darkness ii (Year: 2012, Source: Steam, Similarity: 1.00)

Base Name: bound by flame
Similar Names:
 - bound by flame (Year: 2014, Source: Steam, Similarity: 1.00)
 - bound by flame (Year: 2014.0, Source: VG_Sales, Similarity: 1.00)

Base Name: the evil within
Similar Names:
 - the evil within (Year: 2014.0, Source: VG_Sales, Similarity: 1.00)
 - the evil within (Year: 2014, Source: Steam, Similarity: 1.00)

Base Name: xcom 2
Similar Names:
 - xcom® 2 (Year: 2016, Source: Steam, Similarity: 1.00)

Base Name: dead space™ 2
Similar Names:
 - dead space 2 (Year: 2011.0, Source: VG_Sales, Similarity: 1.00)

Base Name: serious sam 3: bfe
Similar Names:
 - serious sam 3: bfe (Year: 2011.0, Source: VG_Sales, Similarity: 1.00)
 - serious sam 3: bfe (Year: 